<a href="https://colab.research.google.com/github/anushirahatti/mit/blob/master/covid_inpatient_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install apyori

In [0]:
import pandas as pd
import numpy as np

In [3]:
inpatient_cases_data = pd.read_csv("https://raw.githubusercontent.com/anushirahatti/mit/master/inpatient_cases_data.csv")
inpatient_cases_data

,Gender,Age,Clinical classification,No Comorbidities,Cardiovascular and cerebrovascular disease,Endocrine system disease,Malignant tumor,Respiratory system disease,Digestive system disease,Renal disease,Liver disease,Fever,Cough,Chest tightness,Fatigue,Diarrhea,Other Signs and Symptoms,Clinical outcome,COVID-19 RNA clearance,Chest CT findings at discharge
0,Male,47,Heavy,Y,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,N,"Aggravated,turn to ICU",N,Advances
1,Female,53,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,Y,N,N,N,Improved and discharge,Y,Absorption
2,Female,57,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,N,N,N,N,Improved and discharge,Y,No change
3,Male,58,Heavy,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,"Aggravated,transfer to superior hospital",N,Advances
4,Male,54,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,Y,N,N,N,Improved and discharge,Y,Absorption
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Female,30,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,N,N,N,N,Improved and discharge,Y,Absorption
69,Female,59,Ordinary,N,N,Y,N,N,N,N,N,N,Y,N,Y,N,N,Improved and discharge,Y,Absorption
70,Female,87,Ordinary,N,Y,N,N,N,N,N,N,N,Y,N,N,N,N,Improved and discharge,Y,Absorption
71,Female,56,Ordinary,N,Y,N,N,N,N,N,N,Y,Y,N,N,N,N,Improved and discharge,Y,Absorption


In [0]:
#Binning age

def age_group(x):
    if int(x) >= 0 and int(x) < 18:
        x = "Children"
    elif int(x) >= 18 and int(x) < 26:
        x = 'Young Adults'
    elif int(x) >= 26 and int(x) < 41:
        x = 'Adults'
    elif int(x) >= 41 and int(x) < 61:
        x = 'Middle Aged Adults'
    elif int(x) >= 61:
        x = 'Old Adults'
    return x
inpatient_cases_data['Age Group'] = inpatient_cases_data.Age.apply(age_group)
#shark_data.Time_of_Day = shark_data.Time_of_Day.astype('category')

In [5]:
inpatient_cases_data

,Gender,Age,Clinical classification,No Comorbidities,Cardiovascular and cerebrovascular disease,Endocrine system disease,Malignant tumor,Respiratory system disease,Digestive system disease,Renal disease,Liver disease,Fever,Cough,Chest tightness,Fatigue,Diarrhea,Other Signs and Symptoms,Clinical outcome,COVID-19 RNA clearance,Chest CT findings at discharge,Age Group
0,Male,47,Heavy,Y,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,N,"Aggravated,turn to ICU",N,Advances,Middle Aged Adults
1,Female,53,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,Y,N,N,N,Improved and discharge,Y,Absorption,Middle Aged Adults
2,Female,57,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,N,N,N,N,Improved and discharge,Y,No change,Middle Aged Adults
3,Male,58,Heavy,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,"Aggravated,transfer to superior hospital",N,Advances,Middle Aged Adults
4,Male,54,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,Y,N,N,N,Improved and discharge,Y,Absorption,Middle Aged Adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Female,30,Ordinary,Y,N,N,N,N,N,N,N,Y,Y,N,N,N,N,Improved and discharge,Y,Absorption,Adults
69,Female,59,Ordinary,N,N,Y,N,N,N,N,N,N,Y,N,Y,N,N,Improved and discharge,Y,Absorption,Middle Aged Adults
70,Female,87,Ordinary,N,Y,N,N,N,N,N,N,N,Y,N,N,N,N,Improved and discharge,Y,Absorption,Old Adults
71,Female,56,Ordinary,N,Y,N,N,N,N,N,N,Y,Y,N,N,N,N,Improved and discharge,Y,Absorption,Middle Aged Adults


In [6]:
print(inpatient_cases_data.columns)

Index(['Gender', 'Age', 'Clinical classification', 'No Comorbidities',
       'Cardiovascular and cerebrovascular disease',
       ' Endocrine system disease', 'Malignant tumor',
       'Respiratory system disease', 'Digestive system disease',
       'Renal disease', 'Liver disease', 'Fever', 'Cough', 'Chest tightness',
       'Fatigue', 'Diarrhea', 'Other Signs and Symptoms', 'Clinical outcome',
       'COVID-19 RNA clearance', 'Chest CT findings at discharge',
       'Age Group'],
      dtype='object')


In [0]:
for i in inpatient_cases_data.columns:
    inpatient_cases_data[i].replace('Y', i, inplace=True)
    inpatient_cases_data[i].replace('N', np.nan, inplace=True)

In [8]:
inpatient_cases_data

,Gender,Age,Clinical classification,No Comorbidities,Cardiovascular and cerebrovascular disease,Endocrine system disease,Malignant tumor,Respiratory system disease,Digestive system disease,Renal disease,Liver disease,Fever,Cough,Chest tightness,Fatigue,Diarrhea,Other Signs and Symptoms,Clinical outcome,COVID-19 RNA clearance,Chest CT findings at discharge,Age Group
0,Male,47,Heavy,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,Chest tightness,Fatigue,Diarrhea,NaN,"Aggravated,turn to ICU",NaN,Advances,Middle Aged Adults
1,Female,53,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,Chest tightness,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults
2,Female,57,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,No change,Middle Aged Adults
3,Male,58,Heavy,NaN,NaN,Endocrine system disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Aggravated,transfer to superior hospital",NaN,Advances,Middle Aged Adults
4,Male,54,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,Chest tightness,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Female,30,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Adults
69,Female,59,Ordinary,NaN,NaN,Endocrine system disease,NaN,NaN,NaN,NaN,NaN,NaN,Cough,NaN,Fatigue,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults
70,Female,87,Ordinary,NaN,Cardiovascular and cerebrovascular disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Old Adults
71,Female,56,Ordinary,NaN,Cardiovascular and cerebrovascular disease,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults


In [0]:
inpatient_cases_data = inpatient_cases_data.drop(['Age'], axis=1)

In [0]:
for i in inpatient_cases_data.columns:
    inpatient_cases_data[i] = inpatient_cases_data[i].str.strip()

In [11]:
inpatient_cases_data

,Gender,Clinical classification,No Comorbidities,Cardiovascular and cerebrovascular disease,Endocrine system disease,Malignant tumor,Respiratory system disease,Digestive system disease,Renal disease,Liver disease,Fever,Cough,Chest tightness,Fatigue,Diarrhea,Other Signs and Symptoms,Clinical outcome,COVID-19 RNA clearance,Chest CT findings at discharge,Age Group
0,Male,Heavy,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,Chest tightness,Fatigue,Diarrhea,NaN,"Aggravated,turn to ICU",NaN,Advances,Middle Aged Adults
1,Female,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,Chest tightness,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults
2,Female,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,No change,Middle Aged Adults
3,Male,Heavy,NaN,NaN,Endocrine system disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Aggravated,transfer to superior hospital",NaN,Advances,Middle Aged Adults
4,Male,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,Chest tightness,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Female,Ordinary,No Comorbidities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Adults
69,Female,Ordinary,NaN,NaN,Endocrine system disease,NaN,NaN,NaN,NaN,NaN,NaN,Cough,NaN,Fatigue,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults
70,Female,Ordinary,NaN,Cardiovascular and cerebrovascular disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Old Adults
71,Female,Ordinary,NaN,Cardiovascular and cerebrovascular disease,NaN,NaN,NaN,NaN,NaN,NaN,Fever,Cough,NaN,NaN,NaN,NaN,Improved and discharge,COVID-19 RNA clearance,Absorption,Middle Aged Adults


In [0]:
#converting dataframe to list of lists

inpatient_cases_list = []
for i in range(0, 73):
    inpatient_cases_list.append([str(inpatient_cases_data.values[i,j]) for j in range(0, 20)])

In [13]:
inpatient_cases_list

[['Male',
  'Heavy',
  'No Comorbidities',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'Fever',
  'Cough',
  'Chest tightness',
  'Fatigue',
  'Diarrhea',
  'nan',
  'Aggravated,turn to ICU',
  'nan',
  'Advances',
  'Middle Aged Adults'],
 ['Female',
  'Ordinary',
  'No Comorbidities',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'Fever',
  'Cough',
  'Chest tightness',
  'nan',
  'nan',
  'nan',
  'Improved and discharge',
  'COVID-19 RNA clearance',
  'Absorption',
  'Middle Aged Adults'],
 ['Female',
  'Ordinary',
  'No Comorbidities',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'Fever',
  'Cough',
  'nan',
  'nan',
  'nan',
  'nan',
  'Improved and discharge',
  'COVID-19 RNA clearance',
  'No change',
  'Middle Aged Adults'],
 ['Male',
  'Heavy',
  'nan',
  'nan',
  'Endocrine system disease',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'Aggravated,transfer to su

In [0]:
#applying association rules

from apyori import apriori

association_rules = apriori(inpatient_cases_list, min_support=0.0045, min_confidence=0.5, min_lift=3, min_length=3)
association_results = list(association_rules)

In [15]:
print(association_results[0])

RelationRecord(items=frozenset({'Aggravated,transfer to superior hospital', 'Advances'}), support=0.0136986301369863, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Aggravated,transfer to superior hospital'}), items_add=frozenset({'Advances'}), confidence=1.0, lift=14.600000000000001)])


In [16]:
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Streaming output truncated to the last 5000 lines.
Support: 0.0136986301369863
Confidence: 1.0
Lift: 4.866666666666667
Rule: Diarrhea -> COVID-19 RNA clearance
Support: 0.0136986301369863
Confidence: 1.0
Lift: 7.300000000000001
Rule: Diarrhea -> COVID-19 RNA clearance
Support: 0.0136986301369863
Confidence: 1.0
Lift: 10.428571428571429
Rule: Diarrhea -> COVID-19 RNA clearance
Support: 0.0136986301369863
Confidence: 1.0
Lift: 7.300000000000001
Rule: COVID-19 RNA clearance -> Absorption
Support: 0.0136986301369863
Confidence: 0.5
Lift: 6.083333333333334
Rule: COVID-19 RNA clearance -> Absorption
Support: 0.0136986301369863
Confidence: 0.5
Lift: 3.041666666666667
Rule: COVID-19 RNA clearance -> Absorption
Support: 0.0136986301369863
Confidence: 0.5
Lift: 4.055555555555555
Rule: COVID-19 RNA clearance -> Absorption
Support: 0.0136986301369863
Confidence: 0.5
Lift: 3.6500000000000004
Rule: COVID-19 RNA clearance -> Absorption
Support: 0.0136986301369863
Confidence: 0.5
Lift: 6.0833333333333